In [24]:
"""Simple example on how to log scalars and images to tensorboard without tensor ops.

License: Copyleft
"""
__author__ = "Michael Gygli"

import tensorflow as tf
from io import StringIO
import matplotlib.pyplot as plt
import numpy as np

class Logger(object):
    """Logging in tensorboard without tensorflow ops."""

    def __init__(self, log_dir):
        """Creates a summary writer logging to log_dir."""
        self.writer = tf.summary.create_file_writer(log_dir)

    def log_scalar(self, tag, value, step):
        """Log a scalar variable.

        Parameter
        ----------
        tag : basestring
            Name of the scalar
        value
        step : int
            training iteration
        """
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag,
                                                     simple_value=value)])
        self.writer.add_summary(summary, step)

    def log_images(self, tag, images, step):
        """Logs a list of images."""

        im_summaries = []
        for nr, img in enumerate(images):
            # Write the image to a string
            s = StringIO()
            plt.imsave(s, img, format='png')

            # Create an Image object
            img_sum = tf.Summary.Image(encoded_image_string=s.getvalue(),
                                       height=img.shape[0],
                                       width=img.shape[1])
            # Create a Summary value
            im_summaries.append(tf.Summary.Value(tag='%s/%d' % (tag, nr),
                                                 image=img_sum))

        # Create and write Summary
        summary = tf.Summary(value=im_summaries)
        self.writer.add_summary(summary, step)


    def log_histogram(self, tag, values, step, bins=1000):
        """Logs the histogram of a list/vector of values."""
        # Convert to a numpy array
        values = np.array(values)
        print(values[3])
        tfvalues = tf.convert_to_tensor(values, dtype=tf.float64)
        print(tfvalues[3])
        
        tf.summary.histogram(tag, tfvalues, step=step, buckets=5)

        self.writer.flush()



In [72]:
import numpy as np
import random

#logger = Logger('/tmp/test2')
summary_writer = tf.summary.create_file_writer('/tmp/test16')

k, h, w, c = 1, 100, 100, 3
data = tf.random.normal(shape=[k, h, w, c], dtype=tf.float32)
with summary_writer.as_default():
  tf.summary.image('image1', data, step=1)
  #distribution = tf.random.normal((10000,), seed=42, mean=170, stddev=15)
  #tf.random.normal(shape=[3,3,3,32], dtype=tf.float32)
  #mean_moving_normal = tf.random.normal(shape=[1000], mean=(5*5), stddev=1)
  #tf.summary.histogram("test_histogram", mean_moving_normal)
  for i in range(10000):
    tf.summary.scalar('test_scalar',i, step=10000)
    tf.summary.scalar('test_random_scalar',random.randint(1,101), step=10000)
  histdata = tf.random.normal(shape=[h, w], dtype=tf.float64)
  tf.summary.histogram('test_hist',histdata, step=10)


TypeError: __init__() got an unexpected keyword argument 'values'

In [74]:
import tensorflow as tf

with tf.name_scope('hidden') as scope:
  a = tf.constant(5, name='alpha')
  W = tf.Variable(tf.random.uniform([1, 2], -1.0, 1.0), name='weights')
  b = tf.Variable(tf.zeros([1]), name='biases')

In [83]:
import tensorflow as tf
import numpy as np
#sess = tf.InteractiveSession()  #define a session
# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype("float32")
y_data = x_data * 0.1 + 0.3

# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but Tensorflow will
# figure that out for us.)
W = tf.Variable(tf.random.uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b

# Minimize the mean squared errors.
#loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.optimizers.SGD(learning_rate=0.1)
train = optimizer.minimize(loss)

# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()

# Launch the graph.
sess = tf.Session()
sess.run(init)

#### ----> ADD THIS LINE <---- ####
writer = tf.train.SummaryWriter("/tmp/test16", sess.graph)

# Fit the line.
for step in xrange(201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))


NameError: name 'var1' is not defined